In [30]:
from skimage.metrics import mean_squared_error
from sklearn.model_selection import KFold
import lightgbm as lgb #LGB模型
import numpy as np
import pandas as pd

In [31]:
train_data2 = pd.read_csv("./zhengqi_train.txt",sep="\t")
test_data2 = pd.read_csv("./zhengqi_test.txt",sep="\t")

In [32]:
train_data2_f = train_data2[train_data2.columns].values
train_data2_target = train_data2["target"].values

In [33]:
Folds = 5
kf = KFold(n_splits=Folds,shuffle=True,random_state=2021)
MSE_DICT = {"train_mse":[],"test_mse":[]}

In [34]:
# 线下预测模型
for i,(train_index,test_index) in enumerate(kf.split(train_data2)):
    #LGB树模型
    lgb_reg = lgb.LGBMRegressor(
        learning_rate=0.01,
        max_depth=-1,
        n_estimators=5000,
        boosting_type="gbdt",
        random_state=2021,
        objective="regression"
    )

    X_train_KFold, X_test_KFold = train_data2_f[train_index],train_data2_f[test_index]

    y_train_KFold, y_test_KFold = train_data2_target[train_index],train_data2_target[test_index]

    lgb_reg.fit(X = X_train_KFold,
                y = y_train_KFold,
                eval_set=[(X_train_KFold,y_train_KFold),
                          (X_test_KFold,y_test_KFold)],
                eval_names=["Train","Test"],
                early_stopping_rounds=100,
                eval_metric="MSE",
                verbose=50)

    y_train_KFold_predict = lgb_reg.predict(X_train_KFold,num_iteration=lgb_reg.best_iteration_)
    y_test_KFold_predict = lgb_reg.predict(X_test_KFold,num_iteration=lgb_reg.best_iteration_)

    print(f"第{i}轮训练结果：")
    train_mse = mean_squared_error(y_train_KFold_predict,y_train_KFold)
    test_mse = mean_squared_error(y_test_KFold_predict,y_test_KFold)
    print(f"train mse:{train_mse},test mse:{test_mse}")



Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.358459	Test's l2: 0.344019
[100]	Train's l2: 0.131707	Test's l2: 0.127362
[150]	Train's l2: 0.0485491	Test's l2: 0.0475255
[200]	Train's l2: 0.0180527	Test's l2: 0.0181178
[250]	Train's l2: 0.00686335	Test's l2: 0.00724922
[300]	Train's l2: 0.00275103	Test's l2: 0.00322458
[350]	Train's l2: 0.00122732	Test's l2: 0.00173976
[400]	Train's l2: 0.00064804	Test's l2: 0.00116749
[450]	Train's l2: 0.000430771	Test's l2: 0.000942277
[500]	Train's l2: 0.000338008	Test's l2: 0.000857585
[550]	Train's l2: 0.000294812	Test's l2: 0.000820365
[600]	Train's l2: 0.000273748	Test's l2: 0.000803789
[650]	Train's l2: 0.000257831	Test's l2: 0.000792687
[700]	Train's l2: 0.000237535	Test's l2: 0.000783096
[750]	Train's l2: 0.000222792	Test's l2: 0.000787887
[800]	Train's l2: 0.000210743	Test's l2: 0.000793091
Early stopping, best iteration is:
[716]	Train's l2: 0.000232942	Test's l2: 0.000782477
第0轮训练结果：
train mse:0.0002329417